## TODO
1. Проверить sound banks не на линухе, м.б. экспортнуть сразу в гитарпро на винде, т.к. на линухе все играет стандартное пианино.
2. Если экспортировать в drum не получится - перевести все ноты в набор основных звуков: кик, снейр, хэт, крэш и т.д., то, что есть в music21
3. Val loss всегда растет независимо от архитектуры. М.б. стоит избавиться от пауз, которые составляют 40 % партий и добавить конкатенацию с массивом длительностей
4. Или просто добавить больше партий?

In [1]:
import guitarpro
from music21 import instrument, note, stream, chord, duration

import os
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt

from model import NotesRNN

In [2]:
folderpath = "tabs/necro/"

In [3]:
def get_drum_notes(tab):
    
    for track in tab.tracks:
        if track.isPercussionTrack:
            break
    
    notes = []
    for measure in track.measures:
        for beat in measure.voices[0].beats:
            # pass empty measures
            if len(measure.voices[0].beats) == 1 and len(beat.notes) == 0:
                continue
            poly_note = ''
            for note in beat.notes:
                # replace different snare kicks
                if note.value == 40:
                    note.value = 38
                poly_note += str(note.value) + '.'
            notes.append(poly_note)
    
    return notes

In [4]:
notes = []
for filename in os.listdir(folderpath):
    tab = guitarpro.parse(folderpath + filename)
    notes += get_drum_notes(tab)

In [5]:
len(notes)

18233

In [6]:
dictionary = {note: i for i, note in enumerate(set(notes))}
inv_dictionary = {i: note for note, i in dictionary.items()}
notes_to_int = [dictionary[note] for note in notes]
notes_to_int[:10]

[37, 161, 0, 161, 4, 0, 161, 161, 0, 161]

In [7]:
notes[:10]

['46.36.', '42.', '', '42.', '51.38.', '', '42.', '42.', '', '42.']

In [8]:
len(dictionary)

174

In [9]:
def prepare_sequences(notes, dict_size, length, step):
    sequences = []
    targets = []
    for i in range(0, len(notes) - length, step):
        sequences.append(notes[i: i + length])
        targets.append(notes[i + length])
    targets = to_categorical(targets, dict_size)
    return sequences, targets

In [10]:
X, y = prepare_sequences(notes_to_int, len(dictionary), 32, 1)

In [11]:
RNN = NotesRNN(len(dictionary), 100, 128, 4, True, True)

In [12]:
RNN.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    17400       input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, None, 128)    117248      embedding[0][0]                  
__________________________________________________________________________________________________
dropout (Dropout)               (None, None, 128)    0           lstm[0][0]                       
______________________________________________________________________________________________

In [13]:
X = np.array(X)
y = np.array(y)

In [ ]:
RNN.model.fit(
    X, y,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/20
455/455 [==============================] - 27s 52ms/step - loss: 2.9715 - val_loss: 2.6592
Epoch 2/20
455/455 [==============================] - 22s 49ms/step - loss: 2.6876 - val_loss: 2.6036
Epoch 3/20
455/455 [==============================] - 22s 49ms/step - loss: 2.5396 - val_loss: 2.5818
Epoch 4/20
455/455 [==============================] - 22s 49ms/step - loss: 2.4785 - val_loss: 2.5779
Epoch 5/20
455/455 [==============================] - 22s 49ms/step - loss: 2.4317 - val_loss: 2.6125
Epoch 6/20
455/455 [==============================] - 22s 49ms/step - loss: 2.3855 - val_loss: 2.5881
Epoch 7/20
455/455 [==============================] - 22s 49ms/step - loss: 2.3235 - val_loss: 2.5844
Epoch 8/20
455/455 [==============================] - 22s 49ms/step - loss: 2.2844 - val_loss: 2.6156
Epoch 9/20
455/455 [==============================] - 22s 49ms/step - loss: 2.2842 - val_loss: 2.6365
Epoch 10/20
455/455 [==============================] - 22s 49ms/step - loss: 2.230

In [ ]:
plt.figure()
plt.plot(RNN.model.history.history['loss'], label='loss')
plt.plot(RNN.model.history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
def sample(preds, temperature):
    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [ ]:
temperature = 0.5
generated = notes_to_int[:32]

for i in tqdm(range(200)):
    preds = RNN.model.predict(generated[i:])[0]
    generated.append(sample(preds, temperature))

In [ ]:
generated_to_notes = [inv_dictionary[i] for i in generated]
generated_to_notes[32:40]

In [ ]:
midi_stream = stream.Stream()

for gen_notes in generated_to_notes:
    
    gen_notes = gen_notes.split('.')[:-1]
    # chord
    if len(gen_notes) > 1:
        cur_chord = []
        for cur_note in gen_notes:
            new_note = note.Note(int(cur_note))
            new_note.duration = duration.Duration(type='16th')
            new_note.soredInstrument = instrument.Percussion()
            cur_chord.append(new_note)
        midi_stream.append(chord.Chord(cur_chord))
    elif gen_notes != []:
        new_note = note.Note(int(gen_notes[0]))
        new_note.duration = duration.Duration(type='16th')
        new_note.soredInstrument = instrument.Percussion()
        midi_stream.append(new_note)
    else:
        new_note = note.Rest()
        new_note.duration = duration.Duration(type='16th')
        new_note.storedInstrument = instrument.Percussion()
        midi_stream.append(new_note)
        
midi_stream = midi_stream.chordify()

In [ ]:
midi_stream.write('midi', fp=os.path.join('output' + '.mid'))